### Oracle Vector DB wrapped as a llama-index custom Vector Store

* inspired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* updated after **OCI GenAI GA**

In this **sixth demo** we show:
* how to embed a Text using OCI GenAI Embeddings (Cohere V3)
* How to query the Oracle AI Vector Store
* How to create a simplified QA retriever using LlamaIndex
* We have added phoenix traces

In [5]:
import logging
import sys
import numpy as np

from typing import List, Any, Optional, Dict, Tuple
import llama_index
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

# for observability
import phoenix as px

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID

In [6]:
# this is the endpoint after GA, for now Chicago Region
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [7]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.119.1


In [8]:
# phoenix traces
px.launch_app()
llama_index.set_global_handler("arize_phoenix")

2024-03-02 12:53:13,180 - WARNING - Existing running Phoenix instance detected! Shutting it down and starting a new instance...
2024-03-02 12:53:13,430 - WARNING - OpenInferenceTraceCallbackHandler() no longer takes any arguments. The arguments provided has been ignored. For a migration guide, see https://github.com/Arize-ai/phoenix/blob/main/MIGRATION.md


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [9]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for the DB Vector Store

In [10]:
v_store = OracleVectorStore(verbose=True)

In [21]:
question = (
    "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
    # "What is Oracle Data Guard? Can it be used for Disaster recovery"
)

In [12]:
# embed the query using OCI GenAI

query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

In [13]:
np.array(query_embedding)

array([ 0.01473236,  0.03295898,  0.00253105, ...,  0.0045433 ,
        0.034729  , -0.02748108])

#### Use our Vector Store DB

In [14]:
%%time

q_result = v_store.query(query_obj)

2024-03-02 12:53:32,934 - INFO - ---> Calling query on DB
2024-03-02 12:53:33,153 - INFO - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 6 ROWS ONLY
2024-03-02 12:53:33,459 - INFO - Query duration: 0.5 sec.


CPU times: user 22.4 ms, sys: 4.36 ms, total: 26.7 ms
Wall time: 526 ms


#### Displays results

In [15]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Doc. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print(n.metadata)
    print("")

Doc. id: 00b4673ed5c0292f90910328363940db9ae24d2c3ce251327486652b581d5db4
Similarity: 0.693
Part IV Oracle Data Guard Best Practices •Overview of MAA Best Practices for Oracle Data Guard •Plan an Oracle Data Guard Deployment •Configure and Deploy Oracle Data Guard •Tune and Troubleshoot Oracle Data Guard •Monitor an Oracle Data Guard Configuration 
{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}

Doc. id: 9cba3643f9001a783c80e94c93b17ce4dae9bc794d99de7c2e161fea3dd6f151
Similarity: 0.662
Part II Oracle Database High Availability Best Practices •Overview of Oracle Database High Availability Best Practices •Oracle Database Configuration Best Practices •Oracle Flashback Best Practices 
{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}

Doc. id: 654e1cb02d94374751947e659973a253ba94bcc32e7f7c4db6d10e217d86c357
Similarity: 0.652
7 High Availability Data Guard Oracle Data Guard Redo Decryption for Hybrid Disaster Recovery Configurations Oracle Data Guard now provides 

#### Integrate in the RAG picture

In [16]:
# instantiate the client for the LLM
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [17]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [18]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [19]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [22]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

2024-03-02 12:57:20,607 - INFO - ---> Calling query on DB
2024-03-02 12:57:20,775 - INFO - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 5 ROWS ONLY
2024-03-02 12:57:21,049 - INFO - Query duration: 0.4 sec.


Question: What is JSON Relational Duality in Oracle Database 23c? Explain with details

JSON Relational Duality is a feature in Oracle Database 23c that allows developers to access data stored in relational tables in the form of JSON documents. This flexibility is achieved by creating JSON views that overlay the relational data, enabling applications to interact with data in a simpler and more intuitive manner. 

Here are some key details about JSON Relational Duality:

1. **JSON Views**: Duality views are fully updatable JSON views over relational data. These views allow you to present relational data to applications in the JSON format, which is commonly used for data interchange and web development.

2. **Flexibility and Simplicity**: Duality views overcome the challenges of working with relational or document models by providing a simpler and more flexible way to work with data. Developers can use the strengths of both the relational and document models to build applications more ef